# VAE Training

## imports

In [6]:
import os

from models.VAE import VariationalAutoencoder
from utils.loaders import load_mnist

In [7]:
# run params
SECTION = 'vae'
RUN_ID = '0002'
DATA_NAME = 'digits'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #

## data

In [8]:
(x_train, y_train), (x_test, y_test) = load_mnist()

## architecture

In [9]:
vae = VariationalAutoencoder(
    input_dim = (28,28,1)
    , encoder_conv_filters = [32,64,64, 64]
    , encoder_conv_kernel_size = [3,3,3,3]
    , encoder_conv_strides = [1,2,2,1]
    , decoder_conv_t_filters = [64,64,32,1]
    , decoder_conv_t_kernel_size = [3,3,3,3]
    , decoder_conv_t_strides = [1,2,2,1]
    , z_dim = 2
)

if mode == 'build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [10]:
vae.encoder.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 28, 28, 32)   320         encoder_input[0][0]              
__________________________________________________________________________________________________
leaky_re_lu_8 (LeakyReLU)       (None, 28, 28, 32)   0           encoder_conv_0[0][0]             
__________________________________________________________________________________________________
encoder_conv_1 (Conv2D)         (None, 14, 14, 64)   18496       leaky_re_lu_8[0][0]              
____________________________________________________________________________________________

In [11]:
vae.decoder.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 2)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 3136)              9408      
_________________________________________________________________
reshape_2 (Reshape)          (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_1 (Conv2DTran (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 14, 14, 64)        0   

## training

In [12]:
LEARNING_RATE = 0.0005
R_LOSS_FACTOR = 1000

In [13]:
vae.compile(LEARNING_RATE, R_LOSS_FACTOR)

In [14]:
BATCH_SIZE = 32
EPOCHS = 200
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [15]:
vae.train(     
    x_train
    , batch_size = BATCH_SIZE
    , epochs = EPOCHS
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)

Epoch 1/200
60000/60000 [==============================] - 20s 326us/step - loss: 58.1997 - vae_r_loss: 54.9114 - vae_kl_loss: 3.2884

Epoch 00001: saving model to run/vae/0002_digits/weights/weights-001-58.20.h5

Epoch 00001: saving model to run/vae/0002_digits/weights/weights.h5
Epoch 2/200
60000/60000 [==============================] - 16s 269us/step - loss: 51.8592 - vae_r_loss: 47.9641 - vae_kl_loss: 3.8952

Epoch 00002: saving model to run/vae/0002_digits/weights/weights-002-51.86.h5

Epoch 00002: saving model to run/vae/0002_digits/weights/weights.h5
Epoch 3/200
29696/60000 [=============>................] - ETA: 8s - loss: 50.7786 - vae_r_loss: 46.6403 - vae_kl_loss: 4.1383

KeyboardInterrupt: 